# 제주 특산물 가격 예측 AI 경진대회
https://dacon.io/competitions/official/236176/overview/description

## [데이터]
1. train.csv

train 데이터 : 2019년 01월 01일부터 2023년 03월 03일까지의 유통된 품목의 가격 데이터

item: 품목 코드
- TG : 감귤
- BC : 브로콜리
- RD : 무
- CR : 당근
- CB : 양배추

corporation : 유통 법인 코드 -> 법인 A부터 F 존재
location : 지역 코드
- J : 제주도 제주시
- S : 제주도 서귀포시

supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위

2. international_trade.csv

관련 품목 수출입 정보
중량 단위 kg
금액 단위 천 달러


3. test.csv
test 데이터 : 2023년 03월 04일부터 2023년 03월 31일까지의 데이터


4. sample_submission.csv

제출을 위한 양식
2023년 03월 04일부터 2023년 03월 31일까지의 price(원/kg)을 예측
ID는 품목, 유통 법인, 지역 코드로 구성된 식별자

## [리뷰한 코드]-[private 1st] catboost + xgboost + VotingRegressor

In [ ]:
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    # 합쳐서 전처리하기
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)

    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    #년/월/일 추가
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #요일 추가
    df['week_day']=df['timestamp'].dt.weekday

    # 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
    le = LabelEncoder()
    df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

    # 라벨 인코딩
    df["year_month"] = le.fit_transform(df["year_month"])


    # 주차 변수 추가
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    # 주차 누적값
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    # 공휴일 변수 추가
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0

    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))


    # train, test 분리하기
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

## 1. EDA
- 년/월/일/요일/주 파생변수 생성
- 공휴일 변수 추가
- 범주형 변수 : one-hot encoding을 진행함.

## 2. 전처리
- 극 이상치 제거
- 공휴일이지만 안쉬는 날 제외하기


In [ ]:
## 앙상블 모델 정의

cat = CatBoostRegressor(random_state = 2024,
                            n_estimators = 1000,
                            learning_rate = 0.01,
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_notg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_notg["answer"] = pred

answer_notg[["ID","answer"]]

## 3. 모델링
- train, valid, test로 나누어서 진행함.

  (1) catboost+xgboost -> voting을 이용해서 TG 외의 가격 예측

  (2)

      catboost+ xgboost-> voting을 이용함

      catboost -> 예측

        ---> 앙상블: 위 두 모델을 평균으로 하여 앙상블 진행



In [ ]:
# catboost+ xgboost-> voting을 이용함
cat = CatBoostRegressor(random_state = 2024,
                            n_estimators = 1000,
                            learning_rate = 0.01,
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_tg1.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg1["answer"] = np.power(pred,2)

answer_tg1[["ID","answer"]]

In [ ]:
# catboost -> 예측
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024,
                                n_estimators = n_estimators,
                                learning_rate = lrs,
                                depth = max_depths,
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)

cat.fit(Xy2.drop(columns = ["timestamp", "ID","price"]), Xy2["price"])

pred2 = cat.predict(answer_tg2.drop(columns = ["ID"]))
for idx in range(len(pred2)):
    if pred2[idx]<0:
        pred2[idx]= 0
answer_tg2["answer"] = np.power(pred2,2)

answer_tg2[["ID","answer"]]

In [ ]:
#  ---> 앙상블: 위 두 모델을 평균으로 하여 앙상블 진행
total1 = pd.concat([answer_tg1[["ID","answer"]],answer_notg[["ID","answer"]]])
total2 = pd.concat([answer_tg2[["ID","answer"]],answer_notg[["ID","answer"]]])

## [느낀점]

- 예측을 단순 다수결(voting)로 종결하는 방식보다, voting 결과와 개별 모델의 출력을 가중합하여 최종 예측값을 산출하는 방법이 성능 향상에 기여함을 확인하였다.

- 데이터 전처리 과정에서 파생변수 생성 여부와 방법이 최종 성능에 큰 영향을 미친다는 점을 경험하며, 모델링 이전 단계에서의 데이터 가공 과정이 성능 향상의 핵심임을 다시 한 번 느끼게 되었다.